# Bearer Token Injection Pattern for AWS Bedrock AgentCore Gateway Targets

## Overview

Customers want to enable an MCP Client(Cyphr, Amazon Q) to invoke tools (e.g., Asana functions) exposed by backend targets (Lambda functions or RESTful services) through the AgentCore Gateway. A critical requirement is to inject a dynamic bearer token from the MCP client into the target for authentication with the downstream service. The key challenge is the mandatory outbound authentication requirement in the AgentCore Gateway, which can conflict with dynamic header injection.

The following architecture diagram shows the solution we are proposing for injecting auth token (e.g. "Bearer actual-auth-token") without conflicting with AgentCore outbound auth mechanism.

<img src="./images/token-injection-architecture.png" alt="Architecture diagram showing token injection pattern" title="Architecture diagram showing token injection pattern" />

In the solution, the MCP client obtains the auth token to be used by the tool and passes it on to the AgentCore Gateway as a parameter in the payload which is then passed on to the tool as a custom header.

### Deploying the prerequisites

This solution deploys the following components: 
- Cognito user pool for inbound authentication 
- API Gateway with Lambda integration
- API Key for AgentCore outbound authentication


In [4]:
cd prerequisites/agentcore-components

/Users/saurabks/aws_git_hub_repos/amazon-bedrock-agentcore-samples/01-tutorials/02-AgentCore-gateway/04-bearer-token-injection/prerequisites/agentcore-components


In [15]:
!bash prereq.sh

🔍 Getting AWS Account ID...
Region: us-east-1
Account ID: 600406230016
🔧 Starting deployment of infrastructure stack
🚀 Deploying CloudFormation stack: AsanaIntegrationStackInfra

Waiting for changeset to be created..
Waiting for stack create/update to complete
Successfully created/updated stack - AsanaIntegrationStackInfra
✅ Stack AsanaIntegrationStackInfra deployed successfully.
✅ Deployment complete for API stack.
🔧 Starting deployment of infrastructure stack
🚀 Deploying CloudFormation stack: AsanaIntegrationStackCognito

Waiting for changeset to be created..
Waiting for stack create/update to complete
Successfully created/updated stack - AsanaIntegrationStackCognito
✅ Stack AsanaIntegrationStackCognito deployed successfully.
✅ Deployment complete for Cognito stack.
✅ Deployment complete both prerequisite stacks.


## Creating AgentCore Gatewayway with API Gateway as target




### Step 1: Install Dependencies and Import 

Before we start, let's install the pre-requisites for this lab

In [1]:
import os
import sys

# Get the directory of the current script
if "__file__" in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

print(f"current_dir set to {current_dir}")

# Navigate to the directory containing utils.py (two level up)
utils_dir = os.path.abspath(os.path.join(current_dir, "../../.."))

# Add to sys.path
sys.path.insert(0, utils_dir)

import utils

current_dir set to /Users/saurabks/aws_git_hub_repos/amazon-bedrock-agentcore-samples/01-tutorials/02-AgentCore-gateway/04-bearer-token-injection


In [ ]:
# Install required packages
%pip install -U -r requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


### Step 2: Create AgentCore Gateway

The following Python code:
- creates AgentCore Gateway
- adds the previously created APIGateway with API_KEY as Target.

In [5]:
%run agentcore_gateway_creation.py

✅ Fetching AgentCore gateway!
Creating gateway in region us-east-1 with name: agentcore-gw-asana-integration
Found existing gateway with ID: agentcore-gw-asana-integration-zwypthrpiu
✅ Loaded API specification file: [{'openapi': '3.0.0', 'info': {'title': 'Asana Single-Endpoint API', 'version': '1.0.0'}, 'servers': [{'url': 'REPLACE_WITH_API_GATEWAY_URL'}], 'paths': {'/asana': {'post': {'operationId': 'asanaInvoke', 'description': 'Catch-all Asana tool endpoint.', 'parameters': [{'name': 'X-Asana-Token', 'in': 'header', 'required': True, 'schema': {'type': 'string'}, 'description': 'Bearer token for Asana authentication (passed through from agent)'}], 'requestBody': {'required': True, 'content': {'application/json': {'schema': {'type': 'object', 'properties': {'tool_name': {'type': 'string', 'description': 'Name of the Asana operation to execute'}, 'name': {'type': 'string', 'description': 'Task name'}, 'notes': {'type': 'string', 'description': 'Task notes'}, 'project': {'type': 'stri

Let's look at OpenAPI Spec which is used to define the API for this:

<img src="./images/bearer-token-in-api-spec.png" alt="bearer token in api spec as a header" title="bearer token in api spec as a header" />

The token is defined as a header parameter in the schema and forwarded in the HTTP header: ```X-Asana-Token```. 

This is most conventional pattern for authentication tokens.

### Step 3: Test end to end flow

Let's now test the end to end flow where we will invoke ``` tools/list ``` from a MCP client.

In [3]:
import os
import boto3
import time
from botocore.exceptions import ClientError
import utils
import uuid
import json
import requests

# Step 1: Get inbound Auth token for MCP client to authenticate with AgentCore
token_url = utils.get_ssm_parameter("/app/asana/demo/agentcoregwy/cognito_token_url")
client_id = utils.get_ssm_parameter("/app/asana/demo/agentcoregwy/machine_client_id")
client_secret = utils.get_cognito_client_secret()
inbound_auth_token = utils.fetch_access_token(client_id, client_secret, token_url)
print("access token received")

# Step 2: Get Gateway config and prepare payload
GATEWAY_MCP_URL = gateway_url = (
    f"https://{utils.get_ssm_parameter('/app/asana/demo/agentcoregwy/gateway_id')}.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp"
)

SESSION_ID = str(uuid.uuid4())
headers = {
    "Authorization": f"Bearer {inbound_auth_token}",  # For Inbound Auth
    "Content-Type": "application/json",
    "Mcp-Session-Id": SESSION_ID,
}

# Step 3: Call tools/list to get available tools
list_body = {"jsonrpc": "2.0", "id": "list-1", "method": "tools/list"}

list_response = requests.post(GATEWAY_MCP_URL, headers=headers, json=list_body)
print(f"tools/list Status: {list_response.status_code}")
print("Available Tools:")
print(json.dumps(list_response.json(), indent=2))

access token received
tools/list Status: 200
Available Tools:
{
  "jsonrpc": "2.0",
  "id": "list-1",
  "result": {
    "tools": [
      {
        "inputSchema": {
          "type": "object",
          "properties": {
            "tool_name": {
              "description": "Name of the Asana operation to execute",
              "type": "string"
            },
            "workspace": {
              "description": "Workspace GID (optional)",
              "type": "string"
            },
            "notes": {
              "description": "Task notes",
              "type": "string"
            },
            "X-Asana-Token": {
              "description": "Bearer token for Asana authentication (passed through from agent)",
              "type": "string"
            },
            "name": {
              "description": "Task name",
              "type": "string"
            },
            "project": {
              "description": "Project GID",
              "type": "string"
           

### Step 4: Bearer token injection from MCP client to AgentCore Gateway target

Now we will pass the bearer token ``` Bearer ASANA-TOKEN ``` in the parameters (you would replace ``` ASANA-TOKEN ``` with the actual token your MCP client would have obtained) and invoke the tool ```AgentCoreGwyAPIGatewayTarget___asanaInvoke```. 

In [4]:
# Step 4: Call tools/call with Asana token in arguments (will forward as header per updated schema)
call_body = {
    "jsonrpc": "2.0",
    "id": "call-1",
    "method": "tools/call",
    "params": {
        "name": "AgentCoreGwyAPIGatewayTarget___asanaInvoke",  # Exact name from tools/list
        "arguments": {
            "tool_name": "createTask",
            "X-Asana-Token": "Bearer ASANA-TOKEN",  # Actual Asana bearer token here (forwards to header)
            "name": "Test Task from MCP",
            "notes": "This is a test description",
            "project": "your-project-gid",  # Replace with actual Project GID
        },
    },
}

call_response = requests.post(GATEWAY_MCP_URL, headers=headers, json=call_body)
print(f"tools/call Status: {call_response.status_code}")
print("Tool Call Response:")
print(json.dumps(call_response.json(), indent=2))

tools/call Status: 200
Tool Call Response:
{
  "jsonrpc": "2.0",
  "id": "call-1",
  "result": {
    "isError": false,
    "content": [
      {
        "type": "text",
        "text": "{\"message\":\"POST request processed successfully\",\"method\":\"POST\",\"endpoint\":\"/asana\",\"processed\":true,\"received_data\":{\"tool_name\":\"createTask\",\"notes\":\"This is a test description\",\"name\":\"Test Task from MCP\",\"project\":\"your-project-gid\"}}"
      }
    ]
  }
}


### Step 4: Verifying the token injection

Let's verify the token injection by checking the logs in CloudWatch. The following screenshot shows the ``` Bearer token ``` passed from MCP client to AgentCore Gateway to the Lambda function in header ``` X-Asana-Token ``` as defined in the OpenAPI spec.

<img src="./images/Lambda-log-received-headers.png" alt="Bearer token injected as header" title="Bearer token injected as header" />

Rest of the payload from MCP client is available to the Lambda function in the body of the event as shown below:
<img src="./images/Lambda-log-received-body.png" alt="payload body" title="payload body" />

## Clean up

### Delete AgentCore Gateway

In [1]:
import utils
import boto3

gateway_client = boto3.client("bedrock-agentcore-control")

gateway_id = "agentcore-gw-asana-integration-zzszjee4wr"
utils.delete_gateway(gateway_client, gateway_id)

Deleting all targets for gateway agentcore-gw-asana-integration-zzszjee4wr
Deleting target  PY6OTH3EZV
Deleting gateway  agentcore-gw-asana-integration-zzszjee4wr


### Delete Cloudformation stacks

In [ ]:
!bash clean_up.sh